In [40]:
import numpy as np
import tensorly
import pandas as pd
from graspy.utils import * 
import seaborn as sns
import matplotlib.pyplot as plt
import re
import scipy

In [6]:
df = pd.read_excel('SI 3 Synapse lists.xlsx', sheet=0)
new_labels = ['num_contin', 'em_series', 'pre', 'all_post', 'type', 'sections',
 'num_post', 'post1', 'post2', 'post3', 'post4']
df.columns = new_labels
chem = df.loc[df['type'] == 'chemical']
elec = df.loc[df['type'] == 'electrical']
chem_1 = chem.loc[chem['num_post'] == 1]
chem_2 = chem.loc[chem['num_post'] == 2]
chem_3 = chem.loc[chem['num_post'] == 3]
chem_4 = chem.loc[chem['num_post'] == 4]
chem_5 = chem.loc[chem['num_post'] == 5]

,num_contin,em_series,pre,all_post,type,sections,num_post,post1,post2,post3,post4
1174,1047,N2U,RIAL,"SMDDL,obj25487,obj25486,obj25485,obj25484",chemical,4,5,SMDDL,obj25487,obj25486,NaN
1187,1063,N2U,SMDVR,"vBWML6,vBWML2,SMDDL,vBWML3,RMED",chemical,9,5,vBWML6,vBWML2,SMDDL,NaN
3222,3664,N2U,RMDVR,"vBWML1,URAVL.,vBWML5,RMDDL,vBWML4",chemical,6,5,vBWML1,URAVL,vBWML5,NaN
3451,3974,N2U,ADAR,"RIPR,PVR.,AVBL,RIVR,AVJL",chemical,5,5,RIPR,PVR,AVBL,NaN
3722,4314,N2U,RIVR,"GLRVL,GLRVR,vBWMR7,vBWMR6,RIVL",chemical,8,5,GLRVL,GLRVR,vBWMR7,NaN
4971,9892,N2U,AVDL,"AVAL,SABD,AVEL,DA01,VA01",chemical,2,5,AVAL,SABD,AVEL,NaN


In [4]:
df

,num_contin,em_series,pre,all_post,type,sections,num_post,post1,post2,post3,post4
0,653,JSE,PVNL,"PQR,hyp",chemical,1,2,PQR,hyp,NaN,NaN
1,651,JSE,g0,PVCR,chemical,1,1,PVCR,NaN,NaN,NaN
2,652,JSE,g0,PVCR,chemical,1,1,PVCR,NaN,NaN,NaN
3,650,JSE,gx,r30,chemical,1,1,r30,NaN,NaN,NaN
4,2097,JSE,gx,r30,chemical,1,1,r30,NaN,NaN,NaN
5,2121,JSE,gx,r30,chemical,3,1,r30,NaN,NaN,NaN
6,2088,JSE,RID,hyp,chemical,1,1,hyp,NaN,NaN,NaN
7,2089,JSE,RID,hyp,chemical,1,1,hyp,NaN,NaN,NaN
8,2090,JSE,DA09,"DD06,dBWML22",chemical,2,2,DD06,dBWML22,NaN,NaN
9,2109,JSE,PDA,"dBWML22,DD06",chemical,1,2,dBWML22,DD06,NaN,NaN


In [13]:
names = set()
for i, row in chem.iterrows():
    pre = row.pre
    post_names = re.split('[, .]', row.all_post)
    names.add(pre)
    [names.add(post) for post in post_names]
names.remove('')

In [22]:
cell_ids = np.unique(list(names))
inds = range(len(cell_ids))
cell_id_map = dict(zip(cell_ids,inds))

In [45]:
f = np.zeros([len(cell_ids)])
g.nbytes / g.ravel().shape[0]

8.0

In [42]:
f.ravel().shape[0]

449

In [44]:
g = np.zeros(3*[len(cell_ids)])

g.nbytes / g.ravel().shape[0]

8.0

In [52]:
449**4*8/1e9

325.143705608

In [41]:
sparse = scipy.sparse.dok_matrix()

TypeError: __init__() missing 1 required positional argument: 'arg1'

In [ ]:
TENSOR_DIM = 6
def gen_directed_tensor():
    

In [48]:
import tensorly.contrib.sparse

ModuleNotFoundError: No module named 'tensorly.contrib'